In [ ]:
# Import packages andfunctions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.decomposition import PCA
from scipy import stats
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.callbacks import EarlyStopping
import os


In [ ]:
# Read Data from the csv file, FRED-MD dataset
dum = pd.read_csv('/content/2020-05.csv', header=None)
Header = dum.values[:1, 1:]
Data = dum.values[14:-1, 1:].astype('float64')
rawdata = dum.values[2:-1, 1:]
Trans_code = dum.values[1, 1:].reshape(-1, 1)

# Data Transformation
yt = np.nan * np.empty((rawdata.shape[0], rawdata.shape[1]))
for i in range(rawdata.shape[1]):
    x = rawdata[:, i].astype(float)
    tcode = Trans_code[i][0]
    n = x.shape[0]
    small = 1e-6
    if tcode == '1':
        yt[:, i] = x
    elif tcode == '2':
        yt[1:n, i] = x[1:n] - x[0:n-1]
    elif tcode == '3':
        yt[2:n, i] = x[2:n] - 2 * x[1:n-1] + x[0:n-2]
    elif tcode == '4' and np.nanmin(x) > small:
        yt[:, i] = np.log(x)
    elif tcode == '5' and np.nanmin(x) > small:
        x = np.log(x)
        yt[1:n, i] = (x[1:n] - x[0:n-1])
    elif tcode == '6' and np.nanmin(x) > small:
        x = np.log(x)
        yt[2:n, i] = x[2:n] - 2 * x[1:n-1] + x[0:n-2]
    elif tcode == '7':
        z = np.zeros([n])
        z[1:n] = (x[1:n] - x[0:n-1]) / x[0:n-1]
        yt[2:n, i] = z[2:n] - z[1:n-1]

Transformed = yt
Data_tra = Transformed[12:, :]  # (720,128)

# Standardize the dataset
scaler = StandardScaler()
Data_tra_scaled = scaler.fit_transform(Data_tra)


In [ ]:

# Define the autoencoder model
def create_autoencoder(input_dim, encoding_dim):
    model = Sequential()
    model.add(Dense(encoding_dim, activation='relu', input_shape=(input_dim,)))
    model.add(Dense(input_dim, activation='sigmoid'))
    model.compile(optimizer='adam', loss='mse')
    return model

# Hyperparameter tuning using gridsearch with TimeSeriesSplit
def autoencoder_grid_search(X, param_grid):
    def create_model(encoding_dim=64):
        return create_autoencoder(X.shape[1], encoding_dim)

    model = KerasRegressor(build_fn=create_model, verbose=0)
    tscv = TimeSeriesSplit(n_splits=5)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=tscv)
    grid_result = grid.fit(X, X)
    return grid_result


In [ ]:

# defining parameter grid
param_grid = {
    'encoding_dim': [1,2,3,4,5],
    'batch_size': [16, 32, 64],
    'epochs': [50, 100,150]
}

# perform grid search
grid_result = autoencoder_grid_search(Data_tra_scaled, param_grid)

# Print best parameters
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

# Training the final autoencoder with the best parameters
best_params = grid_result.best_params_
autoencoder = create_autoencoder(Data_tra_scaled.shape[1], best_params['encoding_dim'])
early_stopping = EarlyStopping(monitor='loss', patience=10)
autoencoder.fit(Data_tra_scaled, Data_tra_scaled, epochs=best_params['epochs'], batch_size=best_params['batch_size'], callbacks=[early_stopping], verbose=1)

# Extractin the encoded features
encoder = Sequential()
encoder.add(Dense(best_params['encoding_dim'], activation='relu', input_shape=(Data_tra_scaled.shape[1],)))
encoder.compile(optimizer='adam', loss='mse')
encoder.set_weights(autoencoder.get_weights()[:2])
encoded_features = encoder.predict(Data_tra_scaled)

# Save the encoded features into excel file for further use
np.savetxt('/content/Encoded_Features.csv', encoded_features, delimiter=',')


In [ ]:
# Specification of parameters and variables for forecasting
P = 6  # max lags of Y
M = 6  # max lags of factors
K = 6  # max factors

Series = ['CE16OV']  # Series to predict
horstotal = [24]  # Horizons to predict

indexes = [np.where(Header == i)[1][0] for i in Series]  # Series indexes

MSPEs = np.zeros([len(Series), len(horstotal)])  # 16 variables, 7 horizons

est_window = 120
OOS_obs = Data.shape[0] - est_window
TimeSeries = np.zeros([OOS_obs, 2 * len(horstotal)])  # First 7 columns are Ytrue and next 7 are Yhat (for each of 7 horizons)

DMdic = {}



In [ ]:
# Forecasting using autoencoder
DMdic['Autoencoder'] = np.zeros([OOS_obs, len(horstotal) * len(Series)])  # This table stores prediction errors of size (OOS_obs x 1) for each variable and each horizon, this is used to calculate DM test
for varnum, varind in enumerate(indexes):
    for hornum, hor in enumerate(horstotal):
        target = Data[:, varind]
        W = np.delete(encoded_features, varind, 1)

        Yg, Yd = np.zeros([target.shape[0], 1]), np.zeros([target.shape[0], 1])
        Yg[0:hor] = None
        Yg[hor:] = ((np.log(target[hor:]) - np.log(target[:-hor])) / hor).reshape(target[hor:].shape[0], 1)  # Log Diff / h
        Yd[0] = None
        Yd[1:] = (np.log(target[1:]) - np.log(target[:-1])).reshape(target[1:].shape[0], 1)  # Log Diff

        Ytrue = Yg[est_window:]
        Yhat = np.zeros([OOS_obs, 1])

        for t in range(OOS_obs):
            if t < 2:
                kill = 2 - t
            else:
                kill = 0
            Ygt = Yg[t:est_window - hor + 1 + t]
            Ydt = Yd[t:est_window - hor + 1 + t]
            Wt = W[kill + t:est_window - hor + 1 + t, :]
            Wt = Wt[:, ~np.any(np.isnan(Wt), axis=0)]

            Wt = np.unique(Wt.T, axis=0).T
            Wt = stats.zscore(Wt, axis=0, ddof=1)
            T, N = Wt.shape[0], Wt.shape[1]

            allpar = {}
            BIC = np.zeros([K, M, P])
            sigma2 = np.zeros([K, M, P])
            sigma2.fill(np.NaN)
            for pp in range(P):
                [X, Xlag] = lagaugment(Ydt[:-hor], pp + 1)
                X = np.hstack((X[:, 0].reshape(X.shape[0], 1), Xlag[:, 0:-1]))
                X = X[kill:, :]

                for kk in range(K):
                    Fhs = encoded_features[:, :kk + 1]

                    for mm in range(M):
                        [FhsL, FhsLag] = lagaugment(Fhs[:-hor], mm + 1)
                        FhsL = np.hstack((FhsL.reshape(FhsL.shape[0], kk + 1), FhsLag[:, 0:-(kk + 1)]))

                        XF = np.hstack((np.ones([FhsL.shape[0], 1]), np.hstack((X, FhsL))))
                        YXF = np.hstack((Ygt[hor + kill:], XF))
                        YXF = YXF[~np.any(np.isnan(YXF), axis=1), :]

                        beta = np.dot(np.linalg.inv(np.dot(YXF[:, 1:].T, YXF[:, 1:])), np.dot(YXF[:, 1:].T, YXF[:, 0]))

                        allpar[kk, mm, pp] = beta

                        vnum = XF.shape[1]
                        tnum = YXF.shape[0]

                        if tnum <= vnum:  # Handling rare degenerate cases -- makes sure very bad BICs are ignored
                            BIC[kk, mm, pp] = 0
                        else:
                            yin = np.dot(YXF[:, 1:], beta)
                            res = YXF[:, 0] - yin
                            sigma2 = np.dot(res.T, res) / (tnum - vnum)

                            BIC[kk, mm, pp] = np.log(sigma2) + (vnum) * np.log(tnum) / tnum


            [kopt, mopt, popt] = [np.where(BIC == np.min(BIC))[0][0] + 1, np.where(BIC == np.min(BIC))[1][0] + 1, np.where(BIC == np.min(BIC))[2][0] + 1]
            beta = allpar[kopt - 1, mopt - 1, popt - 1]

            # prediction step
            [X, Xlag] = lagaugment(Ydt, popt)
            X = np.hstack((X[:, 0].reshape(X.shape[0], 1), Xlag[:, 0:-1]))
            X = X[-1, :]

            [FhOpt, FhOptlag] = lagaugment(encoded_features[:, :kopt], mopt)
            FhOpt = np.hstack((FhOpt, FhOptlag[:, 0:-kopt]))

            lastx = np.hstack((1, np.hstack((X, FhOpt[-1, :]))))
            Yhat[t] = np.dot(lastx, beta)

        # storing forecasting errors
        print("Variable:", varind, "Mode: Autoencoder", "Horizon:", hor)
        DMdic['Autoencoder'][:, varnum * len(horstotal) + hornum] = Ytrue.reshape(OOS_obs,) - Yhat.reshape(OOS_obs,)
        MSPE = np.mean((Ytrue - Yhat)**2)
        MSPEs[varnum, hornum] = MSPE

# Save MSPEs for autoencoder
np.savetxt('/content/Autoencoder_MSPEs.csv', MSPEs, delimiter=',')

#  Prepare the final table
Autoencoder_MSPEs = pd.read_csv('/content/Autoencoder_MSPEs.csv', header=None)

# Final_table = Autoencoder_MSPEs.copy()
# Final_table.columns = ["h=1", "h=3", "h=6", "h=9", "h=12", "h=18", "h=24"]
# Final_table.index = Series
# Final_table = np.round(Final_table, 4)
# np.savetxt('Data_Out/Final_table_autoencoder.csv', Final_table, delimiter=',')

# Latex = []
# for inum, iseries in enumerate(Series):
#     Latex.append('\multicolumn{8}{c}{' + Series[inum] + ' series}\\\ \n\cmidrule(r){1-8}\n' + Final_table.iloc[inum:inum + 1].to_latex(header=False, index=False)[120:-27] + '\n\cmidrule(r){1-8}\n')
# Latex = "".join(Latex)
# Latex = Latex.replace("S&P", "S\&P")
# print(Latex)

PCA_MSPEs = pd.read_csv('/content/PCA_MSPEs.csv', header=None)

Autoencoder_RMSPE = Autoencoder_MSPEs / PCA_MSPEs.iloc[:, [6]]